# RedShift Challenge

We have a database in our qa-app cluster, ds schema, called dummytable. The challenge is to create a notebook in your sandbox that does the following:
* Read the records from this table and display in a pandas dataframe
* Create a new record in the table, following the format of the other records, then display only the new record in a dataframe
* Update an existing record, then display the updated record in a dataframe
* Delete the new record you created, and display all records in a dataframe to ensure it was deleted.
* Be sure to add some markdown documentation, and inline code documentation to describe what your code is doing.
* Once finished, push up to our data-sci git rep

In [13]:
import boto3 
import json
import psycopg2

In [14]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

CKSM-TJZP
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [15]:
session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
rs_client = boto3.client('redshift', "us-east-1")

endpoint = 'qa-app.ctjussvyafp4.us-east-1.redshift.amazonaws.com'

cluster_credentials = rs_client.get_cluster_credentials(
    ClusterIdentifier = "qa-app",
    DbUser = 'admin',
    DbName = "datascience",
    DbGroups = ["admin_group"],
    AutoCreate = True
)

print(cluster_credentials)

CNXN = psycopg2.connect(
    host = endpoint,
    port = 5439,
    user = cluster_credentials["DbUser"],
    password = cluster_credentials["DbPassword"],
    database = "datascience"
)

{'DbUser': 'IAMA:admin:admin_group', 'DbPassword': 'Ayfd3RwfgcnMH9a5WlTZgH0yekupMlERUt69Up2bqEniOQOxitHiqFSK2Qayb69Jxy8yeJA==', 'Expiration': datetime.datetime(2022, 8, 4, 21, 56, 30, 257000, tzinfo=tzutc()), 'ResponseMetadata': {'RequestId': 'b478aa15-22b6-41ac-9beb-d271cbcc9e24', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b478aa15-22b6-41ac-9beb-d271cbcc9e24', 'content-type': 'text/xml', 'content-length': '492', 'date': 'Thu, 04 Aug 2022 21:41:29 GMT'}, 'RetryAttempts': 0}}
